In [ ]:
from langchain_community.llms import Ollama
from langchain.agents import Tool, AgentExecutor, create_openai_tools_agent
from langchain.agents import initialize_agent, AgentType
import spotipy
from spotipy.oauth2 import SpotifyOAuth




In [ ]:
# Configuração do Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="SEU_CLIENT_ID",
    client_secret="SEU_CLIENT_SECRET",
    redirect_uri="http://localhost:8888/callback",
    scope="user-read-playback-state,user-modify-playback-state"
))

# Ferramentas do Subagente de Música
def play_track(track_name):
    results = sp.search(q=track_name, type='track', limit=1)
    if results['tracks']['items']:
        track_uri = results['tracks']['items'][0]['uri']
        sp.start_playback(uris=[track_uri])
        return f"Tocando agora: {track_name}"
    return "Música não encontrada."

def pause_music():
    sp.pause_playback()
    return "Reprodução pausada."

# Criar o subagente de música
music_tools = [
    Tool(
        name="PlayTrack",
        func=play_track,
        description="Toca uma música específica no Spotify pelo nome."
    ),
    Tool(
        name="PauseMusic",
        func=pause_music,
        description="Pausa a música que está tocando no Spotify."
    )
]

# Conectando o Ollama ao subagente de música
llm_music = Ollama(model="llama2", temperature=0)
music_agent = initialize_agent(music_tools, llm_music, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

# Subagente de Música como Ferramenta do Agente Principal
def music_agent_tool(command):
    return music_agent.run(command)

# Definindo o subagente como ferramenta do agente principal
tools = [
    Tool(
        name="MusicAgent",
        func=music_agent_tool,
        description="Use esta ferramenta para comandos relacionados a músicas, como tocar ou pausar músicas no Spotify."
    )
]

# Agente Principal com o Ollama
llm_main = Ollama(model="llama2", temperature=0)
main_agent = create_openai_tools_agent(llm_main, tools)
agent_executor = AgentExecutor(agent=main_agent, tools=tools)

# Exemplo de uso do agente principal
response = agent_executor.run("Toque a música 'Bohemian Rhapsody' no Spotify.")
print(response)

In [ ]:
audio.export("voz_jarvis_br_final.wav", format="wav")